# Algorithm 1: Billing Model for Retail Markets

## Gather data to one table

In [3]:
import pandas as pd
import numpy as np
import random

profile = pd.read_excel("ele profile.xlsx").T


pro = profile.sample(frac=0.2)
con = profile[~profile.index.isin(pro.index)]

con=con.T

PV = pd.read_excel("PV profile.xlsx").T
WT = pd.read_excel("WT profile.xlsx").T

WT1=WT.iloc[5:10].T
WT2 = WT.iloc[0:5].T
WT_PV_CON = pro.iloc[20:25].T
PV1 = PV.iloc[20:25].T

PV_CON = pro.iloc[0:20].T
PV2 = PV.iloc[0:20].T

only_WT_CON = pro.iloc[25:30].T

# hour = 10

all_list=[]


for hour in range(168):
    statistics = []
    i=5
    
    for j in list(only_WT_CON.columns):
            statistics.append([j,
                only_WT_CON[j][hour],
                0,
                WT1[i][hour],
                WT1[i][hour]-only_WT_CON[j][hour]])
            i+=1
    i=0
    j=20
    z=0
    
    for i in list(WT_PV_CON.columns):

            statistics.append([i,
                WT_PV_CON[i][hour],
                PV1[j][hour] ,
                WT2[z][hour],
                WT2[z][hour]+PV1[j][hour]-WT_PV_CON[i][hour]])
            j+=1
            z+=1
    j=0
    
    for i in list(PV_CON.columns):

        statistics.append([i,
            PV_CON[i][hour],
            PV2[j][hour],
            0,
            PV2[j][hour]-PV_CON[i][hour]])
        j+=1
        
    for i in list(con.columns):
        statistics.append([i,
            con[i][hour],
            0,
            0,
            -con[i][hour]])

    a=pd.DataFrame(statistics, columns=['Client ID','Consumption data','PV data','WT','Consumption/generation profile']
               ,index=[x[0] for x in statistics])
    
    buyer = []
    seller = []
    for i in range(150):
        if statistics[i][4] < 0:
            statistics[i][4] = -statistics[i][4]
            statistics[i].append('buyer')
            buyer.append(statistics[i][0])
        else:
            statistics[i].append('seller')
            seller.append(statistics[i][0])


    b=list(a.index)
    random.shuffle(b)
    perfect = b[0:15]
    good = b[15:105]
    ok = b[105:135]
    bad=b[135:150]

    for i in range(150):

        if i in list(perfect):
            statistics[i].append('perfect')
            statistics[i].append(statistics[i][4])
        if i in list(good):
            statistics[i].append('good')
            statistics[i].append(random.choice([round(statistics[i][4]*random.uniform(0.9,0.99),2),
                                 round(statistics[i][4]*random.uniform(1.01,1.1),2)]))
        if i in list(ok):
            statistics[i].append('ok')
            statistics[i].append(random.choice([round(statistics[i][4]*random.uniform(0.8,0.89),2),
                                 round(statistics[i][4]*random.uniform(1.11,1.2),2)]))
        if i in list(bad):
            statistics[i].append('bad')
            statistics[i].append(random.choice([round(statistics[i][4]*random.uniform(0,0.79),2),
                                 round(statistics[i][4]*random.uniform(1.21,2),2)]))


    buyerl = []
    sellerl = []
    c1 = 0
    c2 = 0

    sorted_stat = []
    sorted_stat = sorted(statistics, key=(lambda x: x[7]))

    for i in range(len(seller)):
        sellerl.append(round(random.uniform(0.06, 0.2), 2))
    for j in range(len(buyer)):
        buyerl.append(round(random.uniform(0.05, 0.19), 2))

    sellerl = sorted(sellerl, reverse=True)
    buyerl = sorted(buyerl)

    for i in range(len(sorted_stat)):
        if sorted_stat[i][5] == 'seller':
            sorted_stat[i].append(sellerl[c1])
            c1 += 1
        else:
            sorted_stat[i].append(buyerl[c2])
            c2 += 1

    statistics = sorted_stat

    now = buyerl[0]
    buyery = []
    buyercount = []
    n = 0
    for i in range(len(buyerl)):
        if now == buyerl[i]:
            n += 1
        else:

            buyercount.append(n)
            buyery.append(now)
            now = buyerl[i]
            n = 1

    buyery.append(now)
    buyery = list(reversed(buyery))

    buyerx = []
    co = 0
    for i in buyery:

        for j in range(len(statistics)):
            if statistics[j][5] == 'buyer':
                if statistics[j][8] == i:
                    co += statistics[j][7]

        buyerx.append(co)

    now = sellerl[0]
    sellery = []
    sellercount = []
    n = 0
    for i in range(len(sellerl)):
        if now == sellerl[i]:
            n += 1
        else:

            sellercount.append(n)
            sellery.append(now)
            now = sellerl[i]
            n = 1

    sellery.append(now)

    sellery = list(reversed(sellery))
    sellerx = []
    co = 0
    for i in sellery:

        for j in range(len(statistics)):
            if statistics[j][5] == 'seller':
                if statistics[j][8] == i:
                    co += statistics[j][7]

        sellerx.append(co)

    c = buyery[2]     #intersecation

    for i in range(len(statistics)):
        if statistics[i][5] == 'buyer':
            if statistics[i][8] > c:
                statistics[i].append('accepted')
            else:
                statistics[i].append('rejected')
        else:
            if statistics[i][8] < c:
                statistics[i].append('accepted')
            else:
                statistics[i].append('rejected')


    statistics.sort()

    all_list.append(statistics)


## Assign supplier to each household

In [ ]:
random.shuffle(all_list[len(all_list)-1])

for i in range(45):
    all_list[len(all_list)-1][i].append("supplier1")
    all_list[len(all_list)-1][i].append(0.2)

for i in range(45, 90):
    all_list[len(all_list)-1][i].append("supplier2")
    all_list[len(all_list)-1][i].append(0.205)

for i in range(90, 120):
    all_list[len(all_list)-1][i].append("supplier3")
    all_list[len(all_list)-1][i].append(0.21)

for i in range(120, 135):
    all_list[len(all_list)-1][i].append("supplier4")
    all_list[len(all_list)-1][i].append(0.215)

for i in range(135, 150):
    all_list[len(all_list)-1][i].append("supplier5")
    all_list[len(all_list)-1][i].append(0.22)


all_list[len(all_list)-1].sort()


for i in range(len(all_list)-1):
    for j in range(len(all_list[i])):
        all_list[i][j].append(all_list[len(all_list)-1][j][10])
        all_list[i][j].append(all_list[len(all_list)-1][j][11])

In [5]:
stat=all_list[1]
a = pd.DataFrame(stat, columns=['Client ID',
                                      'Consumption data',
                                      'PV data',
                                      'WT',
                                      'Consumption/generation profile',
                                      'Role', 
                                      'Prediction algorithm',
                                      'Predicted volume',
                                      'Price offered',
                                      'Bid',
                                      'supplier',
                                      'retail price']
                 , index=[x[0] for x in stat])
a.T.sort_index(axis=1)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Client ID,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Consumption data,0.05,0.05,0.07,0.2,0.14,0.08,0.09,0.16,0.05,0.11,...,0.03,0.05,0.2,0.03,0.1,0.16,0.06,0.16,0.15,0.11
PV data,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WT,0.0,0.0,0.0,0.0,0.27,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.28,0.0,0.0
Consumption/generation profile,0.05,0.05,0.07,0.2,0.13,0.08,0.09,0.16,0.05,0.11,...,0.03,0.05,0.2,0.03,0.1,0.16,0.06,0.12,0.15,0.11
Role,buyer,buyer,buyer,buyer,seller,buyer,buyer,buyer,buyer,buyer,...,buyer,buyer,buyer,buyer,buyer,buyer,buyer,seller,buyer,buyer
Prediction algorithm,good,good,good,ok,good,good,ok,good,bad,bad,...,good,good,good,good,good,good,ok,good,good,ok
Predicted volume,0.05,0.05,0.08,0.17,0.13,0.08,0.11,0.15,0.09,0.14,...,0.03,0.05,0.22,0.03,0.1,0.16,0.07,0.13,0.16,0.1
Price offered,0.07,0.07,0.11,0.18,0.17,0.11,0.14,0.17,0.12,0.16,...,0.06,0.08,0.19,0.06,0.14,0.18,0.11,0.17,0.18,0.14
Bid,rejected,rejected,rejected,accepted,rejected,rejected,rejected,rejected,rejected,rejected,...,rejected,rejected,accepted,rejected,rejected,accepted,rejected,rejected,accepted,rejected


In [6]:
#alg1


supplier_bal=[]

alg1 = all_list
for i in range(len(alg1)):
    supplier_in1,supplier_in2,supplier_in3,supplier_in4,supplier_in5=0,0,0,0,0
    supplier_out1,supplier_out2,supplier_out3,supplier_out4,supplier_out5=0,0,0,0,0
    supplier_bal1,supplier_bal2,supplier_bal3,supplier_bal4,supplier_bal5=0,0,0,0,0

    for j in range(len(alg1[i])):
        Consumer_Bill=0
        Pro_Bill=0

        if alg1[i][j][5] == 'buyer':
            Consumer_Bill = alg1[i][j][4]*alg1[i][j][11]
        else:
            Pro_Bill = alg1[i][j][4]*0.05

        if alg1[i][j][10] == 'supplier1':
            alg1[i][j].append(Consumer_Bill)
            alg1[i][j].append(Pro_Bill)
            supplier_in1 = Consumer_Bill
            supplier_out1 = Pro_Bill
            supplier_bal1 += supplier_in1 - supplier_out1
        if alg1[i][j][10] == 'supplier2':
            alg1[i][j].append(Consumer_Bill)
            alg1[i][j].append(Pro_Bill)
            supplier_in2 = Consumer_Bill
            supplier_out2 = Pro_Bill
            supplier_bal2 += supplier_in2 - supplier_out2
        if alg1[i][j][10] == 'supplier3':
            alg1[i][j].append(Consumer_Bill)
            alg1[i][j].append(Pro_Bill)
            supplier_in3 = Consumer_Bill
            supplier_out3 = Pro_Bill
            supplier_bal3 += supplier_in3 - supplier_out3
        if alg1[i][j][10] == 'supplier4':
            alg1[i][j].append(Consumer_Bill)
            alg1[i][j].append(Pro_Bill)
            supplier_in4 = Consumer_Bill
            supplier_out4 = Pro_Bill
            supplier_bal4 += supplier_in4 - supplier_out4
        if alg1[i][j][10] == 'supplier5':
            alg1[i][j].append(Consumer_Bill)
            alg1[i][j].append(Pro_Bill)
            supplier_in5 = Consumer_Bill
            supplier_out5 = Pro_Bill
            supplier_bal5 += supplier_in5 - supplier_out5
    supplier_bal.append([supplier_bal1,supplier_bal2,supplier_bal3,
                               supplier_bal4,supplier_bal5])

In [12]:
stat=alg1[0]
a = pd.DataFrame(stat, columns=['Client ID',
                                      'Consumption data',
                                      'PV data',
                                      'WT',
                                      'Consumption/generation profile',
                                      'Role',
                                      'Prediction algorithm',
                                      'Predicted volume',
                                      'Price offered',
                                      'Bid',
                                      'supplier',
                                      'retail price',
                                'consumer bills', 'prosumer rewards']
                 , index=[x[0] for x in stat])
a.T.sort_index(axis=1)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Client ID,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Consumption data,0.05,0.05,0.06,0.09,0.09,0.09,0.09,0.14,0.05,0.1,...,0.02,0.05,0.09,0.04,0.11,0.09,0.06,0.11,0.14,0.11
PV data,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WT,0.0,0.0,0.0,0.0,0.29,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.31,0.0,0.0
Consumption/generation profile,0.05,0.05,0.06,0.09,0.2,0.09,0.09,0.14,0.05,0.1,...,0.02,0.05,0.09,0.04,0.11,0.09,0.06,0.2,0.14,0.11
Role,buyer,buyer,buyer,buyer,seller,buyer,buyer,buyer,buyer,buyer,...,buyer,buyer,buyer,buyer,buyer,buyer,buyer,seller,buyer,buyer
Prediction algorithm,good,good,perfect,good,good,perfect,bad,perfect,ok,good,...,good,good,good,good,bad,good,good,bad,good,good
Predicted volume,0.05,0.05,0.06,0.08,0.22,0.09,0.16,0.14,0.06,0.1,...,0.02,0.05,0.09,0.04,0.14,0.09,0.07,0.06,0.14,0.1
Price offered,0.07,0.08,0.09,0.12,0.11,0.13,0.18,0.18,0.09,0.15,...,0.05,0.08,0.14,0.07,0.18,0.15,0.11,0.18,0.18,0.15
Bid,rejected,rejected,rejected,rejected,accepted,rejected,accepted,accepted,rejected,rejected,...,rejected,rejected,rejected,rejected,accepted,rejected,rejected,rejected,accepted,rejected
